In [2]:
import numpy as np
import pandas as pd
from gensim.models import KeyedVectors
from keras.layers import Dense, LSTM, Dropout, Activation
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from pandas.io.json import json_normalize
from sklearn.model_selection import train_test_split

EMBEDDING_DIM = 300
MAX_NUM_WORDS = 10000

# nvidia-docker run -it -p 8888:8888 tensorflow/tensorflow:latest-gpu-py3


/home/dauren/.envs/ML/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
kk_model = KeyedVectors.load_word2vec_format('data/cc.kk.300.vec')

In [26]:
data = pd.read_csv('data/sample.csv')
data = data[:1001]

data = data[data.sentiment != 0]
data['text'] = data['text'].apply(lambda x: x.lower())


In [4]:
# Alem Research News data in russian language
# with open('train.json', encoding='utf-8') as file:
#     train = json.load(file)
#     
# data = json_normalize(train)
# data = data[['text', 'sentiment']]
# data['text'] = data['text'].apply(lambda x: x.lower())


FileNotFoundError: [Errno 2] No such file or directory: 'train.json'

In [27]:
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X)

In [6]:
model = Sequential()
model.add(kk_model.get_keras_embedding())

In [9]:
lstm_out = 128

# model.add(SpatialDropout1D(rate=0.2))
# model.add(LSTM(lstm_out, recurrent_dropout=0.2, dropout=0.2))
# model.add(Dense(2, activation='softmax'))
model.add(Dropout(0.4))
model.add(LSTM(128))
model.add(Dense(64))
model.add(Dropout(0.5))
model.add(Activation('relu'))
model.add(Dense(1))
model.add(Activation('sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 300)         593875500 
_________________________________________________________________
dropout_1 (Dropout)          (None, None, 300)         0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               219648    
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
activation_1 (Activation)    (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 65        
__________

In [7]:
model.add(Dropout(0.4))
model.add(LSTM(128))
model.add(Dense(64))
model.add(Dropout(0.5))
model.add(Activation('relu'))
model.add(Dense(2))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 300)         593875500 
_________________________________________________________________
dropout_1 (Dropout)          (None, None, 300)         0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               219648    
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
activation_1 (Activation)    (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 130       
__________

In [28]:
Y = pd.get_dummies(data['sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33)
print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)


(471, 42) (471, 2)
(232, 42) (232, 2)


In [29]:
batch_size = 64
model.fit(X_train, Y_train, epochs=7, batch_size=batch_size, verbose=2)


Epoch 1/7


 - 0s - loss: 0.6528 - acc: 0.6306


Epoch 2/7


 - 0s - loss: 0.6482 - acc: 0.6667


Epoch 3/7


...

 - 0s - loss: 0.5816 - acc: 0.6964


Epoch 6/7


 - 0s - loss: 0.5673 - acc: 0.7197


Epoch 7/7


 - 0s - loss: 0.5394 - acc: 0.7176


In [30]:
scores = model.evaluate(X_test, Y_test, verbose=2, batch_size=batch_size)
print("Accuracy: %.2f%%" % (scores[1] * 100))


Accuracy: 62.93%


In [12]:
text1 = ["""Көрдік қой, дүние деген иттің көтін, 
Жеп жүр ғой біреуінің біреуі етін, 
Ойлы адамға қызық жоқ бұл жалғанда, 
Көбінің сырты бүтін, іші түтін""".lower()]
tokenized_text = tokenizer.texts_to_sequences(text1)
seq = pad_sequences(tokenized_text, maxlen=28, dtype='int32', value=0)
sentiment = model.predict(seq, batch_size=1, verbose=2)[0]
if np.argmax(sentiment) == 0:
    print("negative")
elif np.argmax(sentiment) == 1:
    print("positive")


positive
